In [2]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data1 = pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx')
with open("best_labels.json", "r") as json_file:
    group_targets = json.load(json_file)
new_column = pd.Series(group_targets["best_labels"], name="组号")
data1 = pd.concat([data1, new_column], axis=1)

In [4]:
data1

,ID,发病到首次影像检查时间间隔,入院首次影像检查流水号,ED_volume.0,随访1流水号,ED_volume.1,随访2流水号,ED_volume.2,随访3流水号,ED_volume.3,...,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8,组号
0,sub001,2.5,9000,48919,29766.0,57898.0,475590.0,81747.0,935055.0,107793.0,...,126558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,sub002,3.0,10800,23526,53724.0,23390.0,249199.0,28415.0,1612859.0,34868.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,sub003,2.0,7200,32621,34281.0,41292.0,142545.0,49767.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,sub004,1.0,3600,22191,61107.0,18956.0,301843.0,9540.0,819004.0,5204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,sub005,5.0,18000,47392,95283.0,23182.0,352595.0,12898.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,sub156,5.5,19800,8968,89575.0,12860.0,452217.0,13716.0,1055942.0,12709.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,sub157,0.5,1800,409,4920.0,3742.0,25620.0,53311.0,72240.0,29545.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,sub158,2.8,10080,17948,66180.0,21099.0,256680.0,44069.0,1111140.0,64243.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,sub159,3.0,10800,14430,95520.0,19019.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 按组号分组
grouped_data = data1.groupby("组号")
grouped_dataframes = {}
for group, group_df in grouped_data:
    grouped_dataframes[group] = group_df.drop(["发病到首次影像检查时间间隔","组号"], axis = 1).reset_index()
    print(grouped_dataframes[group].shape)

(9, 20)
(59, 20)
(11, 20)
(8, 20)
(13, 20)


In [6]:
grouped_dataframes[group]

,index,ID,入院首次影像检查流水号,ED_volume.0,随访1流水号,ED_volume.1,随访2流水号,ED_volume.2,随访3流水号,ED_volume.3,随访4流水号,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8
0,5,sub006,86400,97503,172348.0,95384.0,437874.0,81745.0,619877.0,50859.0,1028332.0,24730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,sub011,3600,5080,61705.0,3642.0,1197272.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41,sub042,7200,12949,93380.0,20132.0,1132337.0,3744.0,2773900.0,625.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63,sub064,3600,7239,29000.0,4015.0,197594.0,1169.0,1123304.0,1736.0,2246219.0,32.0,3458355.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,66,sub067,14400,4242,101930.0,5065.0,459139.0,11560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,69,sub070,10800,8178,34752.0,10374.0,1072648.0,15157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,70,sub071,7200,27830,58536.0,33546.0,161753.0,7538.0,749449.0,66089.0,925171.0,70518.0,3342602.0,487.0,NaN,NaN,NaN,NaN,NaN,NaN
7,71,sub072,7200,10390,70749.0,18574.0,670489.0,16510.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,74,sub075,10800,17633,49198.0,13297.0,311051.0,42038.0,750969.0,34310.0,3339255.0,11202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,85,sub086,10800,3327,66281.0,1154.0,645797.0,6076.0,1253871.0,2490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# # 取XY值
# grouped_dataframes_XY = {}
# for g in grouped_dataframes:
#     x=[]
#     y=[]
#     ID=[]
#     datapp=pd.DataFrame()
#     for i in range(len(list(grouped_dataframes[g].columns))):
#         if list(grouped_dataframes[g].columns)[i][-3:]=='流水号':
#             for j in range(len(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]])):
#                 if j <=99 and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]) and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]):
#                     x.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]/3600)
#                     y.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]/1000)
#                     ID.append(grouped_dataframes[g]['ID'][j])
#     datapp['ID']=ID.copy()
#     datapp['X']=x.copy()
#     datapp['Y']=y.copy()
#     grouped_dataframes_XY[g] = datapp.copy()

In [8]:
#线性
def liner_func(x,a,b):
    return a*x+b

#二次
def erchi_func(x,a,b,c):
    return a*x**2+b*x+c

#三次
def sanchi_func(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d

# 三角函数
def trig_func(x,a,b,c):
    return a*np.sin(x)+b*np.cos(x)+c
# 指数曲线
def target_func(x, a, b, c):
    return a * np.exp(-x / b) + c
# 对数函数
def hyp_func(x, a,b):
    return a*np.log(x)+b

# 定义高斯函数
def gaussian(x, a, b, c):
    return a * np.exp(-(x - b) ** 2 / (2 * c ** 2))

#定义Logistic函数
def logistic(x,a,b,c):
    return  a / (1 + np.exp(-b * (x - c)))


def __sst(y_no_fitting):
    """
    计算SST(total sum of squares) 总平方和
    :param y_no_predicted: List[int] or array[int] 待拟合的y
    :return: 总平方和SST
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_no_fitting]
    sst = sum(s_list)
    return sst


def __ssr(y_fitting, y_no_fitting):
    """
    计算SSR(regression sum of squares) 回归平方和
    :param y_fitting: List[int] or array[int]  拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 回归平方和SSR
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_fitting]
    ssr = sum(s_list)
    return ssr


def __sse(y_fitting, y_no_fitting):
    """
    计算SSE(error sum of squares) 残差平方和
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 残差平方和SSE
    """
    s_list = [(y_fitting[i] - y_no_fitting[i])**2 for i in range(len(y_fitting))]
    sse = sum(s_list)
    return sse


def goodness_of_fit(y_fitting, y_no_fitting):
    """
    计算拟合优度R^2
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 拟合优度R^2
    """
    SSR = __ssr(y_fitting, y_no_fitting)
    SST = __sst(y_no_fitting)
    rr = SSR /SST
    return rr
def selected_func(you,model,model_select):
    maxyou=max(you)
    for s in range(len(you)):
        if you[s]==maxyou:
            
            return model[s],model_select[s]
        
    

def is_non_real(x):
    # 判断复数是否为非实数
    return math.isinf(x) or math.isnan(x)

def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1
        

# 求解方程 f(x) = y
def solve_equation(f,y):
    equation = lambda x: f(x) - y
    x_initial_guess = 0  # 初始猜测值
    x_solution = fsolve(equation, x_initial_guess)
    return x_solution

for g in grouped_dataframes:
    x=[]
    y=[]
    ID=[]
    datapp=pd.DataFrame()
    for i in range(len(list(grouped_dataframes[g].columns))):
        if list(grouped_dataframes[g].columns)[i][-3:]=='流水号':
            for j in range(len(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]])):
                if j <=99 and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]) and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]):
                    x.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]/3600)
                    y.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]/1000)
                    ID.append(grouped_dataframes[g]['ID'][j])
    datapp['ID']=ID.copy()
    datapp['X']=x.copy()
    datapp['Y']=y.copy()
    data=datapp

    Set={}
    youSet={}
    resSet={}
    resTime={}
    you=[]
    model=[]
    model_select=[]
    you_index={}
    RMR=[]
    try:
        a1 = np.polyfit(x, y, 1)#线性
        you1 = goodness_of_fit([liner_func(x[p],a1[0],a1[1]) for p in range(len(x))],y)
        you.append(you1)
        rmr1=[round(liner_func(x[p],a1[0],a1[1])-y[p],4) for p in range(len(x))]
        model.append({'回归类型':'线性回归','回归系数':a1,'残差':rmr1,'ID':ID})
        you_index['线性回归']=you1
        model_select.append((liner_func,a1))
    except:
        pass

    try:
        a2 = np.polyfit(x, y, 2)#二次
        you2 = goodness_of_fit([erchi_func(x[p],a2[0],a2[1],a2[2]) for p in range(len(x))],y)
        rmr2=[round(erchi_func(x[p],a2[0],a2[1],a2[2])-y[p],4) for p in range(len(x))]
        you.append(you2)
        model.append({'回归类型':'二次函数回归','回归系数':a2,'残差':rmr2,'ID':ID})
        you_index['二次函数回归']=you2
        model_select.append((erchi_func,a2))
    except:
        pass

    try:
        a3 = np.polyfit(x, y, 3)#三次
        you3 = goodness_of_fit([sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3]) for p in range(len(x))],y)
        rmr3=[round(sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3])-y[p],4) for p in range(len(x))]
        you.append(you3)
        model.append({'回归类型':'三次函数回归','回归系数':a3,'残差':rmr3,'ID':ID})
        you_index['三次函数回归']=you3
        model_select.append((sanchi_func,a3))
    except:
        pass

    #拟合三角函数模型
    try:
        a4,_=optimize.curve_fit(trig_func,x,y)
        you4 = goodness_of_fit([trig_func(x[p],a4[0],a4[1],a4[2]) for p in range(len(x))],y)
        rmr4=[round(trig_func(x[p],a4[0],a4[1],a4[2])-y[p],4) for p in range(len(x))]
        you.append(you4)
        model.append({'回归类型':'三角函数回归','回归系数':a4,'残差':rmr4,'ID':ID})
        you_index['三角函数回归']=you4
        model_select.append((trig_func,a4))

    except:
        pass

    #拟合指数函数模型
    try:
        a5,_=optimize.curve_fit(target_func,x,y)
        you5 = goodness_of_fit([target_func(x[p], a5[0], a5[1], a5[2]) for p in range(len(x))],y)
        rmr5=[round(target_func(x[p], a5[0], a5[1], a5[2])-y[p],4) for p in range(len(x))]
        you.append(you5)
        model.append({'回归类型':'指数函数回归','回归系数':a5,'残差':rmr5,'ID':ID})
        you_index['指数函数回归']=you5
        model_select.append((target_func,a5))
    except:
        pass

    #拟合对数函数模型
    try:
        a6,_=optimize.curve_fit(hyp_func,x,y)
        you6 = goodness_of_fit([hyp_func(x[p], a6[0],a6[1]) for p in range(len(x))],y)
        rmr6=[round(hyp_func(x[p], a6[0],a6[1])-y[p],4) for p in range(len(x))]
        you.append(you6)
        model.append({'回归类型':'对数函数回归','回归系数':a6,'残差':rmr6,'ID':ID})
        you_index['对数函数回归']=you6
        model_select.append((hyp_func,a6))
    except:
        pass
    try:
        popt, pcov = optimize.curve_fit(gaussian, x, y)
        # 提取拟合参数
        a_fit, b_fit, c_fit = popt

        you7 = goodness_of_fit([gaussian(x[p], a_fit,b_fit,c_fit) for p in range(len(x))],y)
        rmr7=[round(gaussian(x[p], a_fit,b_fit,c_fit)-y[p],4) for p in range(len(x))]
        you.append(you7)
        model.append({'回归类型':'高斯函数回归','回归系数':popt,'残差':rmr7,'ID':ID})
        you_index['高斯函数回归']=you7
        model_select.append((gaussian,popt))
    except:
        pass

    try:
        popt, pcov = optimize.curve_fit(logistic, x, y)
        # 提取拟合参数
        a_fit, b_fit, c_fit = popt
        you8 = goodness_of_fit([logistic(x[p], a_fit,b_fit,c_fit) for p in range(len(x))],y)

        rmr8=[round(logistic(x[p], a_fit,b_fit,c_fit)-y[p],4) for p in range(len(x))]
        you.append(you8)
        model.append({'回归类型':'logistic函数回归','回归系数':popt,'残差':rmr8,'ID':ID})
        you_index['logistic函数回归']=you8
        model_select.append((logistic,popt))

    except:
        pass

    # model=[f'线性回归，系数为{a1}',f'二次函数回归，系数为{a2}',f'三次函数回归，系数为{a3}',f'三角函数回归，系数为{a4}',f'指数函数回归，系数为{a5}',f'对数函数回归，系数为{a6}']
    res,model_=selected_func(you,model,model_select)

    # Set[index]=res#选择函数模型，例如：1用三角函数
    # youSet[index]=you_index#拟合优度

    print(you_index)
    p=0 #取最大
    for i in range(len(list(you_index.keys()))):
        if you_index[list(you_index.keys())[i]]>you_index[list(you_index.keys())[p]]:
            p=i

    print(model[p])
    ID=model[p]['ID']
    RMR=model[p]['残差']
    SET_RMR={}
    for i in range(len(ID)):
        SET_RMR[ID[i]]=[]
    for i in range(len(ID)):
        SET_RMR[ID[i]].append(model[p]['残差'][i])
    datasave=pd.DataFrame(columns=['ID','残差'])
    datasave['ID']=SET_RMR.keys()
    datasave['残差']=SET_RMR.values()
    datasave.to_excel(f'Q22填表答案_向量结果_{g}.xlsx',index=False)
    datasave['残差']=datasave['残差'].apply(lambda z: sum(z) / len(z))
    datasave.to_excel(f'Q22填表答案_单值结果_{g}.xlsx',index=False)

{'线性回归': 0.1335282096839314, '二次函数回归': 0.16332792603604024, '三次函数回归': 0.2890009860085615, '三角函数回归': 0.08239520726311755, '指数函数回归': 0.13352745192484927, '对数函数回归': 0.004498664797687777, '高斯函数回归': 0.32350421244582755, 'logistic函数回归': 1.7396204247444496e-18}
{'回归类型': '高斯函数回归', '回归系数': array([ 37.48057775, 334.96486405, 272.92110842]), '残差': [-5.6389, -4.4631, -29.3452, -1.0973, 2.5535, 5.4369, -6.2018, 7.2605, 17.5417, -4.5446, 0.0556, -3.396, -1.2855, 1.8492, 0.1004, -19.4562, 2.527, 17.6077, -5.0841, 15.0052, 12.8076, -25.463, 6.181, 4.1509, -24.6167, 2.4702, 20.3626, -0.4689, 29.4808, 2.7418, -28.9861, 9.3728, 13.2383, 0.6038, 9.3331, 6.9098, 0.0002, -23.2503, 0.0, -0.9955], 'ID': ['sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub018', 'sub019', 'sub020', 'sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub018', 'sub019', 'sub020', 'sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub018', 'sub019', 'sub020', 'sub002', 'sub004', 'sub012', 'sub018', 